In [ ]:
# ==========================================
# IMPORTAR LIBRERÍAS
# ==========================================
import pandas as pd
import numpy as np


# ==========================================
# CARGAR DATOS DESDE CSV + LIMPIEZA
# ==========================================

ruta_sales = "C:\Users\miguelcastuera\Desktop\sales.csv"
ruta_inventories = "C:\Users\miguelcastuera\Desktop\inventories.csv"
ruta_satisfaction = "C:\Users\miguelcastuera\Desktop\satisfaction.csv"

df_sales = pd.read_csv(ruta_sales)
df_inventories = pd.read_csv(ruta_inventories)
df_satisfaction = pd.read_csv(ruta_satisfaction)

print("=== Cabeceras originales ===")
print("Sales:")
display(df_sales.head())
print("\nInventories:")
display(df_inventories.head())
print("\nSatisfaction:")
display(df_satisfaction.head())

df_sales = df_sales.dropna()
df_inventories = df_inventories.dropna()
df_satisfaction = df_satisfaction.dropna()

print("\n=== Dimensiones después de dropna() ===")
print("Sales:", df_sales.shape)
print("Inventories:", df_inventories.shape)
print("Satisfaction:", df_satisfaction.shape)


# ==========================================
# VENTAS TOTALES POR PRODUCTO Y POR TIENDA
# ==========================================

df_sales["Ingreso"] = df_sales["Cantidad_Vendida"] * df_sales["Precio_Unitario"]

ventas_prod_tienda = df_sales.groupby(["ID_Tienda", "Producto"])["Cantidad_Vendida"].sum().reset_index()
ventas_prod_tienda.rename(columns={"Cantidad_Vendida": "Cantidad_Total_Vendida"}, inplace=True)

print("=== Ventas totales por producto y tienda ===")
display(ventas_prod_tienda.head())

ingresos_tienda = df_sales.groupby("ID_Tienda")["Ingreso"].sum().reset_index()
ingresos_tienda.rename(columns={"Ingreso": "Total_Ventas"}, inplace=True)

print("\n=== Ingresos totales por tienda ===")
display(ingresos_tienda)


# ==========================================
# RESUMEN ESTADÍSTICO DE LAS VENTAS
# ==========================================

print("=== Resumen estadístico de Cantidad_Vendida ===")
display(df_sales["Cantidad_Vendida"].describe())

print("\n=== Resumen estadístico de Ingreso por línea ===")
display(df_sales["Ingreso"].describe())

print("\n=== Resumen estadístico de Total_Ventas por tienda ===")
display(ingresos_tienda["Total_Ventas"].describe())


# ==========================================
# ANÁLISIS DE INVENTARIOS: ROTACIÓN
# ==========================================

ventas_prod_tienda = df_sales.groupby(["ID_Tienda", "Producto"])["Cantidad_Vendida"].sum().reset_index()
ventas_prod_tienda.rename(columns={"Cantidad_Vendida": "Cantidad_Total_Vendida"}, inplace=True)

df_inv_ventas = pd.merge(
    df_inventories,
    ventas_prod_tienda,
    on=["ID_Tienda", "Producto"],
    how="left"
)

df_inv_ventas["Cantidad_Total_Vendida"] = df_inv_ventas["Cantidad_Total_Vendida"].fillna(0)

df_inv_ventas["Rotacion"] = df_inv_ventas["Cantidad_Total_Vendida"] / df_inv_ventas["Stock_Disponible"]

df_inv_ventas["Porcentaje_Ventas"] = df_inv_ventas["Cantidad_Total_Vendida"] / df_inv_ventas["Stock_Disponible"]

print("=== Inventarios con rotación calculada ===")
display(df_inv_ventas.head())

criticos = df_inv_ventas[df_inv_ventas["Porcentaje_Ventas"] < 0.10]

print("\n=== Filas con inventario crítico (Porcentaje_Ventas < 10%) ===")
display(criticos)

tiendas_criticas = criticos["ID_Tienda"].unique()
print("\nTiendas con niveles críticos de inventario:")
print(tiendas_criticas)


# ==========================================
# SATISFACCIÓN DEL CLIENTE
# ==========================================

df_ventas_satisfaccion = pd.merge(
    ingresos_tienda,
    df_satisfaction,
    on="ID_Tienda",
    how="left"
)

print("=== Ventas y satisfacción por tienda ===")
display(df_ventas_satisfaccion)

tiendas_baja_satisfaccion = df_ventas_satisfaccion[
    df_ventas_satisfaccion["Satisfacción_Promedio"] < 60
]

print("\n=== Tiendas con baja satisfacción (< 60%) ===")
display(tiendas_baja_satisfaccion)


# ==========================================
# CÁLCULOS CON NUMPY: MEDIANA Y DESVIACIÓN
# ==========================================

ventas_array = ingresos_tienda["Total_Ventas"].to_numpy()

print("Array de ventas totales por tienda:")
print(ventas_array)

mediana_ventas = np.median(ventas_array)
desviacion_ventas = np.std(ventas_array)

print("\n=== Estadísticos con Numpy ===")
print(f"Mediana de las ventas totales: {mediana_ventas:.2f} €")
print(f"Desviación estándar de las ventas totales: {desviacion_ventas:.2f} €")


# ==========================================
# SIMULACIÓN DE PROYECCIONES FUTURAS (NUMPY)
# ==========================================

np.random.seed(42)

num_tiendas = len(ventas_array)
num_meses = 12

factores = 0.9 + 0.3 * np.random.rand(num_tiendas, num_meses)

proyecciones = ventas_array.reshape(-1, 1) * factores

print("=== Proyeccionses de ventas (primeras tiendas) ===")
print(proyecciones[:2, :])

media_proy = proyecciones.mean()
mediana_proy = np.median(proyecciones)
desv_proy = np.std(proyecciones)

print("\n=== Estadísticas de las proyecciones ===")
print(f"Media: {media_proy:.2f} €")
print(f"Mediana: {mediana_proy:.2f} €")
print(f"Desviación estándar: {desv_proy:.2f} €")


=== Cabeceras originales ===
Sales:


,ID_Tienda,Producto,Cantidad_Vendida,Precio_Unitario,Fecha_Venta
0,1,Producto A,20,100,2023-01-05
1,1,Producto B,15,200,2023-01-06
2,2,Producto A,30,100,2023-01-07
3,2,Producto C,25,300,2023-01-08
4,3,Producto A,10,100,2023-01-09



Inventories:


,ID_Tienda,Producto,Stock_Disponible,Fecha_Actualización
0,1,Producto A,50,2023-01-05
1,1,Producto B,40,2023-01-06
2,2,Producto A,60,2023-01-07
3,2,Producto C,45,2023-01-08
4,3,Producto A,30,2023-01-09



Satisfaction:


...


Tiendas con niveles críticos de inventario:
[]
=== Ventas y satisfacción por tienda ===


,ID_Tienda,Total_Ventas,Satisfacción_Promedio,Fecha_Evaluación
0,1,5000,85,2023-01-15
1,2,10500,90,2023-01-15
2,3,9000,70,2023-01-15
3,4,13000,65,2023-01-15
4,5,13000,55,2023-01-15



=== Tiendas con baja satisfacción (< 60%) ===


,ID_Tienda,Total_Ventas,Satisfacción_Promedio,Fecha_Evaluación
4,5,13000,55,2023-01-15


Array de ventas totales por tienda:
[ 5000 10500  9000 13000 13000]

=== Estadísticos con Numpy ===
Mediana de las ventas totales: 10500.00 €
Desviación estándar de las ventas totales: 2973.21 €
=== Proyecciones de ventas (primeras tiendas) ===
[[ 5061.81017827  5926.07145961  5597.99091272  5397.9877263
   4734.02796066  4733.9917805   4587.12541825  5799.26421866
   5401.67251761  5562.10886669  4530.87674144  5954.86477824]
 [12072.19431852 10118.86819864 10022.7486467  10027.72420604
  10408.36306532 11102.98275964 10810.62680872 10367.37179162
  11377.33661838  9889.40566105 10370.25564289 10604.03980638]]

=== Estadísticas de las proyecciones ===
Media: 10487.73 €
Mediana: 10633.02 €
Desviación estándar: 3197.61 €
